# Telecomunicaciones: Scraping de URLs legítimas

**Fecha de ejecución:** 22/07/2025  
**Empresas objetivo:**  
Movistar, Vodafone, Orange, MásMóvil, Yoigo, Pepephone, Digi, Euskaltel

---


| Empresa     | URLs únicas | Fecha      | Estado / Observaciones                |
|-------------|------------|------------|---------------------------------------|
| Vodafone    |   106      | 22/07/2025 | OK                                    |
| Yoigo       |    55      | 22/07/2025 | OK                                    |
| Pepephone   |    54      | 22/07/2025 | OK                                    |
| MásMóvil    |    49      | 22/07/2025 | OK                                    |
| Movistar    |    48      | 22/07/2025 | OK                                    |
| Digi        |    46      | 22/07/2025 | OK                                    |
| Euskaltel   |    43      | 22/07/2025 | OK                                    |
| Orange      |    29      | 22/07/2025 | OK                                    |


In [1]:
# Importamos las librerías necesarias para el scraping y el análisis de datos
import requests                             # Para hacer peticiones HTTP a las webs
from bs4 import BeautifulSoup               # Para parsear el HTML de la página web
from urllib.parse import urlparse, urljoin  # Para trabajar con URLs y dominios
import pandas as pd                         # Para almacenar y exportar los datos en DataFrame
import time                                 # Para pausar entre peticiones y evitar bloqueos


In [6]:
def obtener_urls(base_url, delay=1):
    """
    Descarga la página principal de una web y extrae todas las URLs internas.
    Solo añade URLs que pertenezcan al dominio principal o subdominios de la empresa.
    - base_url: URL principal de la empresa
    - delay: segundos de espera tras cada request para evitar bloqueos
    """
    try:
        # Definimos un User-Agent "realista" para que el servidor no bloquee la petición por bot
        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.124 Safari/537.36"
            )
        }
        # Hacemos la petición HTTP a la web (con timeout de 10 segundos)
        response = requests.get(base_url, timeout=10, headers=headers)

        # Parseamos el HTML recibido con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraemos el dominio principal (ej: 'amazon.es' de 'www.amazon.es')
        dominio_empresa = urlparse(base_url).netloc
        if dominio_empresa.startswith('www.'):
            dominio_base = dominio_empresa[4:]
        else:
            dominio_base = dominio_empresa

        # Creamos un set para guardar solo URLs únicas
        urls = set()

        # Buscamos todos los enlaces <a> que tengan atributo href
        for link in soup.find_all('a', href=True):
            href = link['href']                  # Obtenemos el href
            href = urljoin(base_url, href)       # Convertimos a URL absoluta
            dominio_href = urlparse(href).netloc # Dominio del enlace

            # Eliminamos 'www.' para comparar dominios correctamente
            if dominio_href.startswith('www.'):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añadimos solo si es del dominio principal o subdominio
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Esperamos el tiempo indicado (para no ser bloqueados por el servidor)
        time.sleep(delay)

        # Devolvemos la lista de URLs y un estado "OK"
        return list(urls), "OK"

    except Exception as e:
        # Si hay error, lo devolvemos como estado y lista vacía de URLs
        return [], str(e)


In [5]:
empresas_telecom = {
    "Movistar": "https://www.movistar.es",
    "Vodafone": "https://www.vodafone.es",
    "Orange": "https://www.orange.es",
    "MásMóvil": "https://www.masmovil.es",
    "Yoigo": "https://www.yoigo.com",
    "Pepephone": "https://www.pepephone.com",
    "Digi": "https://www.digimobil.es",
    "Euskaltel": "https://www.euskaltel.com"
}


In [7]:
# Lista donde almacenaremos los resultados de todas las empresas
resultados = []

# Guardamos la fecha actual 
fecha = pd.Timestamp.today().strftime("%d/%m/%Y")

# Iteramos sobre cada empresa y su URL
for nombre, url_base in empresas_telecom.items():
    print(f'Scraping empresa: {nombre}')                # Log para saber el progreso
    urls, estado = obtener_urls(url_base)               # Llamamos a la función de scraping
    if urls:
        # Si hay URLs, añadimos cada una como fila en los resultados
        for url in urls:
            resultados.append({
                'empresa': nombre,
                'url': url,
                'fecha': fecha,
                'estado': estado
            })
    else:
        # Si falla o no hay URLs, registramos igualmente el intento
        resultados.append({
            'empresa': nombre,
            'url': '',
            'fecha': fecha,
            'estado': estado
        })

# Convertimos la lista de resultados en un DataFrame de pandas
df_scrap = pd.DataFrame(resultados)

# Guardamos el DataFrame como CSV en la carpeta correspondiente (cambia el nombre por sector)
df_scrap.to_csv('../data/raw/teleco_legitimas_crudo.csv', index=False)

# Mostramos por pantalla las primeras filas del DataFrame para revisión rápida
print(df_scrap.head())


Scraping empresa: Movistar
Scraping empresa: Vodafone
Scraping empresa: Orange
Scraping empresa: MásMóvil
Scraping empresa: Yoigo
Scraping empresa: Pepephone
Scraping empresa: Digi
Scraping empresa: Euskaltel
    empresa                                               url       fecha  \
0  Movistar  https://www.movistar.es/tarifas-moviles/roaming/  22/07/2025   
1  Movistar       https://www.movistar.es/compramos-tu-movil/  22/07/2025   
2  Movistar                     https://comunidad.movistar.es  22/07/2025   
3  Movistar      https://www.movistar.es/tv/agenda-deportiva/  22/07/2025   
4  Movistar                https://www.movistar.es/tv/futbol/  22/07/2025   

  estado  
0     OK  
1     OK  
2     OK  
3     OK  
4     OK  


In [8]:
print(df_scrap['empresa'].value_counts())




empresa
Vodafone     106
Yoigo         55
Pepephone     54
MásMóvil      49
Movistar      48
Digi          46
Euskaltel     43
Orange        29
Name: count, dtype: int64


- **Empresas scrapeadas:** 8
- **Total URLs obtenidas:** 430
- **Observaciones:**
    - Todas las webs han permitido scraping básico y han devuelto una cantidad razonable de URLs.
    - Vodafone destaca con el mayor número de enlaces únicos; Orange es la que menos expone.
    - Ningún error ni bloqueo detectado, lo que indica poca protección anti-bot en las homes de las telecos.
- **Siguientes pasos:**
    - Si interesa, scrapear páginas internas de Orange para ampliar cobertura.
    - Continuar con el siguiente sector (Energía / Utilities).
